In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
tbd.__version__

'0.3.6'

In [30]:
#写一个从grids得到params的功能
bounds = [113.6, 22.4, 113.605, 22.405]
params = (113.6, 22.4, 0.004863669213934598, 0.004496605206422906)

grid,params = tbd.rect_grids(bounds,500)
tbd.regenerate_params(grid)

(113.60000000000001,
 22.400000000000002,
 0.004863669213932553,
 0.004496605206423254)

In [43]:
def regenerate_params(grid):
    '''
    Generate gridding params from grid.

    Parameters
    -------
    grid : GeoDataFrame
        grids generated by transbigdata 
                                               

    Returns
    -------
    params : List
        Gridding parameters (lonStart,latStart,deltaLon,deltaLat), lonStart and latStart are the lower-left coordinates, deltaLon, deltaLat are the length and width of a single grid

    Examples
    -------
    >>> import transbigdata as tbd
    >>> bounds = [113.6, 22.4, 113.605, 22.405]
    >>> grid,params = tbd.rect_grids(bounds,500)
    >>> tbd.grid_to_params(grid)
    (113.6, 22.4, 0.004863669213934598, 0.004496605206422906)
    '''
    loncol_unique = grid.drop_duplicates(subset = ['LONCOL']).copy()
    latcol_unique = grid.drop_duplicates(subset = ['LATCOL']).copy()
    if (len(latcol_unique)==1)|(len(loncol_unique)==1):
        raise Exception('Generate params failed, you must ensure that there are multiple LONCOL and LATCOL in the grids')
    loncol_unique['LONCOL_gap'] = loncol_unique['LONCOL'].shift(-1)-loncol_unique['LONCOL']
    loncol_unique['x'] = loncol_unique['geometry'].apply(lambda r:r.exterior.coords[0][0])
    loncol_unique['x_gap'] = loncol_unique['x'].shift(-1)-loncol_unique['x']
    deltaLon = (loncol_unique['x_gap']/loncol_unique['LONCOL_gap']).iloc[:-1].mean()

    latcol_unique['LATCOL_gap'] = latcol_unique['LATCOL'].shift(-1)-latcol_unique['LATCOL']
    latcol_unique['x'] = latcol_unique['geometry'].apply(lambda r:r.exterior.coords[0][1])
    latcol_unique['x_gap'] = latcol_unique['x'].shift(-1)-latcol_unique['x']
    deltaLat = (latcol_unique['x_gap']/latcol_unique['LATCOL_gap']).iloc[:-1].mean()

    a = grid.iloc[:1]
    lonstart = (a.centroid.x-a['LONCOL'].iloc[0]*deltaLon).iloc[0]
    latstart = (a.centroid.y-a['LATCOL'].iloc[0]*deltaLat).iloc[0]
    params = (lonstart,latstart,deltaLon,deltaLat)
    return params

In [38]:
#写一个从params和shape得到grids的功能
grid

,LONCOL,LATCOL,HBLON,HBLAT,geometry
0,0,0,113.600000,22.400000,"POLYGON ((113.59757 22.39775, 113.60243 22.397..."
1,0,1,113.600000,22.404497,"POLYGON ((113.59757 22.40225, 113.60243 22.402..."
2,1,0,113.604864,22.400000,"POLYGON ((113.60243 22.39775, 113.60730 22.397..."
3,1,1,113.604864,22.404497,"POLYGON ((113.60243 22.40225, 113.60730 22.402..."


In [27]:
bounds = [113.6, 22.4, 113.705, 22.505]
params = (113.6, 22.4, 0.004863669213934598, 0.004496605206422906)

def grid_from_params(params,location):
    '''
    Generate grids from params and bounds or shape.

    Parameters
    -------
    params : List
        Gridding parameters (lonStart,latStart,deltaLon,deltaLat), lonStart and latStart are the lower-left coordinates, deltaLon, deltaLat are the length and width of a single grid
    location : bounds(List) or shape(GeoDataFrame)
        Where to generate grids.
                                               
    Returns
    -------
    grid : GeoDataFrame
        grids generated by transbigdata 
    
    '''
    if type(location)==list:
        shape = ''
        bounds = location
    elif type(location)==gpd.geodataframe.GeoDataFrame:
        shape = location
        bounds = shape.unary_union.bounds
    else:
        raise Exception('Location should be either bounds(List) or shape(GeoDataFrame)')
    loncolstart,latcolstart = tbd.GPS_to_grids(bounds[0],bounds[1],params)
    loncolend,latcolend = tbd.GPS_to_grids(bounds[2],bounds[3],params)
    loncolstart,latcolstart,loncolend,latcolend
    grid = []
    for i in range(loncolstart,loncolend+1):
        for j in range(latcolstart,latcolend+1):
            grid.append([i,j])
    grid = gpd.GeoDataFrame(grid,columns = ['LONCOL','LATCOL'])
    grid['HBLON'],grid['HBLAT'] = tbd.grids_centre(grid['LONCOL'],grid['LATCOL'],params)
    grid['geometry'] = tbd.gridid_to_polygon(grid['LONCOL'],grid['LATCOL'],params)
    if type(shape) !=gpd.geodataframe.GeoDataFrame:
        return grid
    else:
        grid.crs = shape.crs
        grid = gpd.sjoin(grid,shape)
        return grid

In [28]:
shape = gpd.read_file(r'../example/data/sz.json')
bounds = shape.unary_union.bounds
grids = grid_from_params(params,shape)

In [34]:
bounds = [113.6, 22.4, 113.605, 22.405]
params = (113.6, 22.4, 0.004863669213934598, 0.004496605206422906)
list(grid_from_params(params,bounds)['geometry'].iloc[3].exterior.coords)

[(113.60243183460696, 22.402248302603212),
 (113.6072955038209, 22.402248302603212),
 (113.6072955038209, 22.406744907809635),
 (113.60243183460696, 22.406744907809635),
 (113.60243183460696, 22.402248302603212)]